#  <center> Taller  de Aprendizaje Automático </center>
##  <center> Taller 7: Gatos y Perros  </center>

Con el resurgimiento de las redes convolucionales en 2012, entre otras cosas gracias a la viabilidad de entrenar con millones de imágenes, una serie de problemas considerados muy difíciles o inabordables hasta entonces debido a la necesidad de generar algoritmos muy específicos para cada tarea  pasaron a estar al alcance de cualquier desarrollador. Un ejemplo emblemático es el de distinguir entre imágenes de perros y gatos. Debido a la gran variabilidad intra clase de estos animales, es realmente dificil generar características que permitan separar bien las clases y probablemente el esfuerzo que conlleva generar características específicas, no esté justificado por la relevancia del problema. En este taller se verá el procedimiento para entrenar un clasificador a partir de un número no muy grande de imágenes, sin necesidad de diseñar características.

## Objetivos

 - Familiarizarse con la estructura *Dataset* de tensorflow 
 - Aprender a utilizar modelos de clasificación de imágenes preentrenados en *tensorflow* con imágenes de *Imagenet* 
 - Conocer las opciones para realizar *aumento de datos* y aplicárselo a las imágenes de perros y gatos
 - Realizar *transferencia de aprendizaje* del modelo preentrenado con *Imagenet* al problema de clasificar perros y gatos
 - Explicar en base a qué regiones de una imagen el clasificador toma la decisión 

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/TAA-fing/TAA-2022/blob/main/talleres/taller7_gatos_y_perros.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Ejecutar en Google Colab</a>
  </td>
</table>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
import matplotlib.pyplot as plt

## Parte 0 - Utilización de GPU

El entrenamiento de redes convolucionales en general es más rápido cuando se utiliza GPU (Graphic Processor Unit). Correr la siguiente celda y verificar que se está utilizando una. Debería observar un mensaje similar al siguiente:

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

En caso de no observar dicho mensaje y estar trabajando en Colab, verifique que el entorno de ejecución es el correcto, vaya a:  
Entorno de ejecuccióón-->Cambiar tipo de entorno de ejecución-->GPU

In [ ]:
tf.config.list_physical_devices('GPU')

## Parte 1 - Levantar los datos

Se trabajará con los datos disponibles en [Kaggle](https://www.kaggle.com/c/dogs-vs-cats/overview). En caso de estar trabajando en Colab seguir los siguientes pasos:

a) Installar el paquete *kaggle* para acceder a los datos 

In [ ]:
!pip install kaggle

b) Realizar la configuración necesaria para obtener datos desde la plataforma Kaggle. Para ello deberá ir a la página de la competencia y en la sección *data* aceptar los términos. Luego ejecutar la siguiente celda y pasarle el *token* de su usuario (ver comentario en celda).

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from google.colab import files

# El archivo solicitado es para habilitar la API de Kaggle en el entorno que está trabajando.
# Este archivo se descarga entrando a su perfíl de Kaggle, en la sección API, presionando donde dice: Create New API Token

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
#Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Una vez guardado el *token* se pueden descargar los datos.

In [ ]:
# celda para bajar los datos desde colab
!kaggle competitions download -c dogs-vs-cats
!unzip /content/dogs-vs-cats.zip
!unzip /content/train.zip
!unzip /content/test1.zip

## Parte 2 - Preparar los datos

En este taller en vez de almacenar los datos en un arreglo de numpy como en talleres anteriores, los almacenaremos en un *Dataset* de Tensorflow. Para ello se utilizará la función `tf.keras.preprocessing.image_dataset_from_directory()`. Dicha función asume que en el directorio que se pasa como parámetro hay tantos subdirectorios como clases hay en el problema y que dentro de cada subdirectorio están las imágenes correspondientes a esa clase. Los datos originalmente no están almacenados siguiendo esta estructura. Utilizar la función `preparar_datos_entrenamiento()` para generar una estructura de archivos adecuada para  `tf.keras.preprocessing.image_dataset_from_directory()`. Luego utilizar esta última para generar un Dataset de entrenamiento y uno de validación con las siguientes características.

*   80% de entrenamiento y 20% de validación
*   Batch size 32
*   Tamaño de imagen 180

[Aquí](https://keras.io/examples/vision/image_classification_from_scratch/) un ejemplo de keras que muestra cómo hacerlo.


In [ ]:
def preparar_datos_entrenamiento(data_folder, output_folder):
    
    '''
    Genera una estructura de directorios adecuada para la función image_dataset_from_directory()
    Entrada: 
       data_folder: directorio raíz en que se encuentran los datos descargados desde kaggle
       output_folder: directorio en que se guardaran los datos reorfanizados
    '''
    
    classes = ['cat','dog']
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    for clase in classes:
        class_folder = os.path.join(output_folder, clase)
        if not os.path.exists(class_folder):
            os.makedirs(class_folder)
    
    train_folder = os.path.join(data_folder, 'train')
    imgs_paths = os.listdir(train_folder)
    print('Hay %d imágenes de entrenamiento' % len(imgs_paths))
    for img_name in imgs_paths:
        img_class = 'cat' if img_name.startswith('cat') else 'dog'
       
        source_img_path = os.path.join(train_folder, img_name)
        dst_img_path = os.path.join(output_folder,img_class, img_name)
        if os.path.exists(dst_img_path):
            os.unlink(dst_img_path)
       
        os.symlink(source_img_path, dst_img_path)
    
    return 
            

## Parte 3 - Exploración de datos

Mostrar algunos ejemplos de los datos de entrenamiento (un batch por ejemplo). ¿En qué formato se encuentran almacenados los datos? ¿Son adecuados para entrenar una red neuronal?

## Parte 4 - Aumento de datos

**4a)** Siguiendo con el ejemplo de keras, realizar aumento de datos a la primer imagen del dataset y mostrar los resultados.      
**4b)** Además de las opciones que se muestan el ejemplo ¿Qué otras opciones de aumento de datos brinda Keras? Muestre alguna que no aparezca en el ejemplo.      
**4c)** Asegúrese de entender por qué en algún caso puede tener sentido realizar el aumento de datos en el modelo y en otros sobre el dataset.  

## Parte 5 - Entrenando con aumento de datos en el modelo

Copie el modelo del ejemplo y entrene durante 10 épocas utilizando dicho modelo.

**5a)** Asegúrese que entiende cómo fue construída la red y cada uno de los bloques que se utilizan      
**5b)** Modifique la salida de la red para que la salida tenga *num_classes* unidades independientemente de cuál sea el número de clases.         
**5c)** Entrene la red durante 10 épocas          

## Parte 6 - Utilizando Modelos Preentrenados

En esta parte en vez de entrenar un modelo desde cero vamos a utilizar un modelo preentrenado en otro problema y adaptarlo al de clasificar entre perros y gatos. Se evaluarán dos modelos: Resnet50 y Xception.

**6a)** Levantar ambos modelos con los pesos preentrenados con la base *Imagenet*. Ver un ejemplo en la sección *Using Pretrained Models from Keras* del *Capíítulo 14* del libro del curso.  
**6b)** Adaptar el ejemplo del libro para mostrar, para cada una de los modelos cargados, cuáles son las *top3* clases de un *batch* de imágenes del dataset de entrenamiento. ¿Qué observa?  

## Parte 7 - Visualización de filtros

En el caso del modelo *Resnet50* mostrar los filtros aprendidos en la primera capa de convolución. Explique con algún ejemplo por qué puede ser razonable utilizar estos mismos filtros en otro problema.

## Parte 8 - Transferencia de aprendizaje

Tomando como referencia la sección *Pretrained Models for Transfer Learning*  del *Capíítulo 14* del libro y el ejemplo [Transfer Learning](https://keras.io/guides/transfer_learning/) de keras, realizar transferencia de aprendizaje utilizando ambos modelos. 

## Parte 9 - Grad Cam

Una vez que se entrena una red convolucional es muy útil entender en qué basa la red sus predicciones. Uno de los métodos propuestos en ese sentido es [GradCAM](https://arxiv.org/pdf/1610.02391.pdf). 

**9a)** ¿Para qué puede ser útil un método que explique los resultados obtenidos?     
**9b)** Adaptar el [ejemplo](https://keras.io/examples/vision/grad_cam/) de keras y mostrar sobre algunas imágenes de validación las predicciones realizadas y los mapas de activación superpuestos a la imagen. Para eso utilizar (o adaptar) las funciones `make_gradcam_heatmap()` y `save_and_display_gradcam()`.

## Parte 10 - Evauar con el conjunto de Test

Evaluar el modelo con que haya obtenido mejores resultados con el conjunto de validación con el conjunto de test. En este taller el foco no está puesto en optimizar los clasificadores, sin embargo verifique que los resultados obtenidos con el conjunto de test son del orden de los obtenidos con el conjunto de valdiación.